In [1]:
from datetime import date, timedelta
import mlflow
import numpy as np
import os
from pathlib import Path
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pprint import pprint
import sys

path_to_analysis = Path(os.getcwd()).parent / 'Analysis'
sys.path.append(str(path_to_analysis))

from dataPrep import Config, DataPrediction

#### MLFlow Einstellungen definieren.

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("VorhersageModell")

artifact_path = "bereitschaftsdienstplans"

#### Vorhersagemodell in Form eines MLFlow-Modells erstellen. 

In [3]:
class VorhersageModell(mlflow.pyfunc.PythonModel):
    
    def __init__(self):
        super().__init__()
        self.model_adabooreg = self.load_sklearn_model(model_name='AdaBooReg', version=2)
        self.model_trend_reg = self.load_sklearn_model(model_name='TrendRegression', version=1)
        self.model_reg_calls_demand = self.load_sklearn_model(model_name='RegressionCallsDemand', version=1)
        self.startdate = np.datetime64('2016-04-01')
        self.tup_features = ('month', 'dayofmonth', 'weekday', 'weekofyear', 'dayofyear', 'season')

    #def load_context(self, context):
        
    def load_sklearn_model(self, model_name, version):
        
        model_uri = f"models:/{model_name}/{version}"
        
        # Load model as a PyFuncModel.
        loaded_model = mlflow.sklearn.load_model(model_uri=model_uri)

        return loaded_model

    def predict(self, context, model_input) -> pd.DataFrame:
        
        date = np.array([np.datetime64(date) for date in model_input])

        calls_reg_pred = VorhersageModell._date_list_in_reg_out(date, self.startdate, self.model_trend_reg)

        df_date = pd.DataFrame(date, columns=['date'])

        df_pred = VorhersageModell.add_date_features(df_date, self.tup_features)

        df_pred_2 = df_pred[list(self.tup_features)]

        adabooreg_pred = self.model_adabooreg.predict(df_pred_2).reshape(-1)

        pred = calls_reg_pred + adabooreg_pred

        sby_plus_duty = self.model_reg_calls_demand.predict(pred.reshape(-1, 1))

        # Subtrahieren von n_duty, vorausgestetzt n_duty ist 1900
        sby_need = np.round(((sby_plus_duty - 1900).reshape(-1)), 0).astype(int)

        s_sby_need = pd.Series(sby_need, name='sby_need_pred')

        df_pred = pd.concat([df_pred['date'], s_sby_need], axis=1)

        return df_pred

    @staticmethod
    def _date_list_in_reg_out(date: list['datetime64'],
                              startdate: 'datetime64',
                              model: 'sklearn-LinearRegression'
                             ) -> np.array:

        # Regression für Notrufe
        reg_trend = model
        x = np.array((date-startdate).astype('timedelta64[D]') + np.timedelta64(1, 'D'))

        # numpy array aus series date
        x = x.reshape(-1, 1).astype(int)
        calls_reg_pred = reg_trend.predict(x).reshape(-1)

        return calls_reg_pred

    @staticmethod
    def add_date_features(df: pd.DataFrame, features: tuple[str], date_column_name: str='date') -> pd.DataFrame:
        df_features = df.copy()
        c = df_features[date_column_name]

        if 'month' in features:
            df_features['month'] = c.dt.month # Monat als Zahl (1-12)

        if 'year' in features:
            df_features['year'] = c.dt.year # Jahr (4-stellig)
        
        if 'dayofmonth' in features:
            df_features['dayofmonth'] = c.dt.day # Tag des Monats (1-31)

        if 'weekday' in features:
            # Wochentag als Zahl (Montag = 0, Sonntag = 6)
            df_features['weekday'] = c.dt.weekday

        if 'weekofyear' in features:
            # Kalenderwoche als Zahl (1-52)
            df_features['weekofyear'] = c.dt.isocalendar().week

        if 'dayofyear' in features:
            # Tag des Jahres als Zahl (1-365)
            df_features['dayofyear'] = c.dt.dayofyear
        
        # Datum des 15. des vorherigen Monats
        df_features['predict_day'] = c - DateOffset(months=1, day=15)

        # Anzahl der Tage seit dem ersten Tag im Datensatz
        df_features['day'] = (c - pd.Timestamp('2016-04-01')).dt.days + 1

        if 'season' in features:
            m = df_features['month']
            # Jahreszeit als Zahl (1-4) (1=Winter, 2=Frühling, 3=Sommer, 4=Herbst)
            df_features['season'] = (m-1) % 12 // 3 + 1

        if 'calls' in df_features:
            df_features['status'] = 'actual'
        else:
            df_features['status'] = 'prediction'

        return df_features

#### MLFlow-Model speichern

In [4]:
# Initiate the MLflow run context
with mlflow.start_run(run_name='4') as run:

    # trainiertes Modell-Instanz protokollieren
    mlflow.pyfunc.log_model(artifact_path=artifact_path, python_model=VorhersageModell())

mlflow.end_run()

C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\FsME\Prog\TDSP-BsdEf\BsdEf_venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### MLFlow-Model vom Server laden

In [5]:
def load_model(model_name, version):
    
    model_uri = f"models:/{model_name}/{version}"
    
    # Load model as a PyFuncModel.
    loaded_model = mlflow.pyfunc.load_model(model_uri=model_uri)

    return loaded_model

model_vorhersage = load_model(model_name='Vorhersagemodell', version=6)

#### Vorhersagedaten definieren

In [6]:
test_date = ['2019-05-27', '2019-05-28', '2019-05-29', '2019-05-30', '2019-05-31']

#### Vorhersagen machen

In [7]:
model_vorhersage.predict(test_date)

,date,sby_need_pred
0,2019-05-27,-51
1,2019-05-28,-48
2,2019-05-29,-57
3,2019-05-30,-88
4,2019-05-31,-113
